In [1]:
import turicreate as tc

In [2]:
products=tc.SFrame("amazon_baby.csv")

Finished parsing file /root/amazon_baby.csv

Parsing completed. Parsed 100 lines in 0.676558 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,str,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 105270 lines. Lines per second: 141972

Finished parsing file /root/amazon_baby.csv

Parsing completed. Parsed 183531 lines in 0.927645 secs.

In [3]:
products.head()

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4


## Build the word count vector for each review




In [4]:
products["word_count"]=tc.text_analytics.count_words(products["review"])

In [5]:
products.head()

name,review,rating,word_count
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3,"{'handles': 1.0,'stripping': 1.0, ..."
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5,"{'recommend': 1.0,'highly': 1.0, ..."
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5,"{'quilt': 1.0, 'of': 1.0,'the': 1.0, 'than': 1.0, ..."
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5,"{'tool': 1.0, 'clever':1.0, 'approach': 2.0, ..."
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5,"{'rock': 1.0, 'many':1.0, 'headaches': 1.0, ..."
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5,"{'thumb': 1.0, 'or': 1.0,'break': 1.0, 'trying': ..."
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4,"{'for': 1.0, 'barnes':1.0, 'at': 1.0, 'is': ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5,"{'right': 1.0, 'because':1.0, 'questions': 1.0, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5,"{'like': 1.0, 'and': 1.0,'changes': 1.0, 'the': ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4,"{'in': 1.0, 'pages': 1.0,'out': 1.0, 'run': 1.0, ..."


In [6]:
tc.visualization.set_target()

In [7]:
products["name"].show()

Materializing SArray

## Explore vulli sophie

In [8]:
giraffe_reviews=products[products["name"]=="Vulli Sophie the Giraffe Teether"]

In [9]:
len(giraffe_reviews)

785

In [10]:
giraffe_reviews["rating"].show()

Materializing SArray

## Build a sentiment classifier

In [11]:
products["rating"].show()

Materializing SArray

## Define what is positive and a negative sentiment

In [12]:
#ignore all three stars
products=products[products["rating"]!=3]

In [13]:
#positive sentiment =4 star or 5 star
products["sentiments"]=products["rating"]>=4

In [14]:
products.head()

name,review,rating,word_count,sentiments
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5,"{'recommend': 1.0,'highly': 1.0, ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5,"{'quilt': 1.0, 'of': 1.0,'the': 1.0, 'than': 1.0, ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5,"{'tool': 1.0, 'clever':1.0, 'approach': 2.0, ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5,"{'rock': 1.0, 'many':1.0, 'headaches': 1.0, ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5,"{'thumb': 1.0, 'or': 1.0,'break': 1.0, 'trying': ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4,"{'for': 1.0, 'barnes':1.0, 'at': 1.0, 'is': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5,"{'right': 1.0, 'because':1.0, 'questions': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5,"{'like': 1.0, 'and': 1.0,'changes': 1.0, 'the': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4,"{'in': 1.0, 'pages': 1.0,'out': 1.0, 'run': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4,"{'tracker': 1.0, 'now':1.0, 'its': 1.0, 'sti ...",1


##Let's train the sentiment classifier


In [15]:
train_data,test_data=products.random_split(.8,seed=0)

In [16]:
sentiment_model=tc.logistic_classifier.create(train_data,target="sentiments",features=["word_count"],validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 1

...

| 2         | 11       | 2.669978  | 3.027383     | 0.938650          | 0.916466            |

| 3         | 12       | 2.669978  | 3.370694     | 0.927417          | 0.901814            |

| 4         | 14       | 1.326072  | 3.817823     | 0.945200          | 0.918088            |

| 9         | 20       | 1.326072  | 5.718929     | 0.977654          | 0.917698            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

## Evaluate the sentiment model

In [17]:
sentiment_model.evaluate(test_data,metric="roc_curve")

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+--------------------+--------------------+-------+------+
 | threshold |        fpr         |        tpr         |   p   |  n   |
 +-----------+--------------------+--------------------+-------+------+
 |    0.0    |        1.0         |        1.0         | 27976 | 5328 |
 |   1e-05   | 0.847972972972973  | 0.9975693451529882 | 27976 | 5328 |
 |   2e-05   | 0.829954954954955  | 0.9971761509865599 | 27976 | 5328 |
 |   3e-05   | 0.818506006006006  | 0.9969616814412353 | 27976 | 5328 |
 |   4e-05   | 0.8109984984984985 | 0.9967472118959108 | 27976 | 5328 |
 |   5e-05   | 0.8057432432432432 | 0.9966042321990277 | 27976 | 5328 |
 |   6e-05   | 0.7991741741741741 | 0.9962825278810409 | 27976 | 5328 |
 |   7e-05   | 0.7952327327327328 | 0.9961752931083786 | 27976 | 5328 |
 |   8e-05   | 0.7920420420420421 | 0.9961038032599371 | 27976 | 5328 |
 |   9e-05   | 0.7882882882882

## Apply the learned model to understand the giraffe

In [18]:
giraffe_reviews["predicted_sentiments"]=sentiment_model.predict(giraffe_reviews,output_type="probability")

In [19]:
giraffe_reviews

name,review,rating,word_count,predicted_sentiments
Vulli Sophie the GiraffeTeether ...,He likes chewing on allthe parts especially the ...,5,"{'purchase': 1.0,'teething': 1.0, ...",0.9993655365682543
Vulli Sophie the GiraffeTeether ...,My son loves this toy andfits great in the diaper ...,5,"{'a': 1.0, 'is': 1.0,'when': 1.0, 'him': 1.0, ...",0.9998633791689672
Vulli Sophie the GiraffeTeether ...,There really should be alarge warning on the ...,1,"{'made': 1.0, 'of': 1.0,'packaging': 1.0, 'no': ...",0.25452681974918556
Vulli Sophie the GiraffeTeether ...,All the moms in my moms'group got Sophie for ...,5,"{'another': 1.0, 'out':1.0, 'run': 1.0, 'lost': ...",0.9165688083895903
Vulli Sophie the GiraffeTeether ...,I was a little skepticalon whether Sophie was ...,5,"{'disappointed': 1.0,'will': 1.0, 'take': ...",0.6855768205778686
Vulli Sophie the GiraffeTeether ...,I have been reading aboutSophie and was going ...,5,"{'late': 1.0, 'perfect':1.0, 'pack': 1.0, 'on ...",0.99999994452112
Vulli Sophie the GiraffeTeether ...,My neice loves her sophieand has spent hours ...,5,"{'delight': 1.0, 'in':1.0, 'other': 1.0, ...",0.9979351181092762
Vulli Sophie the GiraffeTeether ...,What a friendly face!And those mesmerizing ...,5,"{'inside': 1.0, 'water':1.0, 'don': 1.0, 'up': ...",0.9999745004834412
Vulli Sophie the GiraffeTeether ...,We got this just for myson to chew on instea ...,5,"{'its': 1.0, 'fine': 1.0,'is': 1.0, 'which': 1.0, ...",0.9460144428329712
Vulli Sophie the GiraffeTeether ...,"My baby seems to likethis toy, but I could ...",3,"{'off': 1.0, 'have': 2.0,'of': 1.0, 'some': 1.0, ...",0.38301136141334013


## Sort the reviews based on the predicted sentiment and explore

In [20]:
giraffe_reviews=giraffe_reviews.sort("predicted_sentiments",ascending=False)

In [21]:
giraffe_reviews.head()

name,review,rating,word_count,predicted_sentiments
Vulli Sophie the GiraffeTeether ...,I'll be honest...I boughtthis toy because all the ...,4,"{'around': 1.0,'explore': 1.0, 'they': ...",1.0
Vulli Sophie the GiraffeTeether ...,As a mother of 16monthold twins; I bought ...,5,"{'will': 1.0, '15months':1.0, 'would': 2.0, ...",1.0
Vulli Sophie the GiraffeTeether ...,"Sophie, oh Sophie, yourtime has come. My ...",5,"{'11': 1.0, 'prisrob':1.0, '12': 1.0, 'who': ...",1.0
Vulli Sophie the GiraffeTeether ...,We got this littlegiraffe as a gift from a ...,5,"{'out': 1.0, 'would':1.0, 've': 1.0, 'enou ...",0.9999999999998377
Vulli Sophie the GiraffeTeether ...,"As every mom knows, youalways want to give your ...",5,"{'whether': 1.0,'neutral': 1.0, 'gend ...",0.9999999999998284
Vulli Sophie the GiraffeTeether ...,My Mom-in-Law boughtSophie for my son whe ...,5,"{'penny': 1.0, 'little':1.0, 'perfect': 1.0, ...",0.9999999999997958
Vulli Sophie the GiraffeTeether ...,"My 4 month old son isteething, and I've tried ...",4,"{'worth': 1.0, 'works':1.0, 'teether': 1.0, ...",0.9999999999994914
Vulli Sophie the GiraffeTeether ...,Let me just start off byaddressing the choking ...,5,"{'question': 1.0, 'must':1.0, 'overall': 1.0, ...",0.9999999999941254
Vulli Sophie the GiraffeTeether ...,I'm not sure why Sophieis such a hit with the ...,4,"{'makers': 1.0, 'or':1.0, 'take': 1.0, 'can': ...",0.999999999987423
Vulli Sophie the GiraffeTeether ...,"I admit, I didn't getSophie the Giraffe at ...",4,"{'dye': 1.0, 'of': 1.0,'cause': 1.0, 'fade': ...",0.9999999999829475


In [22]:
giraffe_reviews[2]["review"]

"Sophie, oh Sophie, your time has come. My granddaughter, Violet is 5 months old and starting to teeth. What joy little Sophie brings to Violet. Sophie is made of a very pliable rubber that is sturdy but not tough. It is quite easy for Violet to twist Sophie into unheard of positions to get Sophie into her mouth. The little nose and hooves fit perfectly into small mouths, and the drooling has purpose. The paint on Sophie is food quality.Sophie was born in 1961 in France. The maker had wondered why there was nothing available for babies and made Sophie from the finest rubber, phthalate-free on St Sophie's Day, thus the name was born. Since that time millions of Sophie's populate the world. She is soft and for babies little hands easy to grasp. Violet especially loves the bumpy head and horns of Sophie. Sophie has a long neck that easy to grasp and twist. She has lovely, sizable spots that attract Violet's attention. Sophie has happy little squeaks that bring squeals of delight from Viol

## Show negative review

In [23]:
giraffe_reviews[-2]["review"]


'I wanted to love this product and was excited to buy it when I became pregnant but am now hesitant to let my baby use it after reading about the recall in Europe. Apparently, as I understand it, their toxin standards of measurement are lower than ours so they have not been recalled here (apparently we are OK with low levels of nitrates in the toys our children put in their mouths, but Europeans are not...hmmm)...Be that as it may, toxins registering even CLOSE to a dangerous level made me nervous about using. After digging around online I did discover that the company claims to have changed the product after a certain date and lists manufacturing codes so you can check yours (those listed were made after a certain date and are said to be safer). Sadly mine was not made after the &#34;improved&#34; date but I could not return it because there was no formal recall in our country. I considered returning it and hunting for one with an approved manufacturing date but man that was just too 

## ASSIGNMENT


In [48]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']
for word in selected_words:
    products[word] = products['review'].apply(lambda s : s.split().count(word))

In [26]:
products

name,review,rating,word_count,sentiments,awesome
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5,"{'recommend': 1.0,'highly': 1.0, ...",1,0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5,"{'quilt': 1.0, 'of': 1.0,'the': 1.0, 'than': 1.0, ...",1,0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5,"{'tool': 1.0, 'clever':1.0, 'approach': 2.0, ...",1,0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5,"{'rock': 1.0, 'many':1.0, 'headaches': 1.0, ...",1,0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5,"{'thumb': 1.0, 'or': 1.0,'break': 1.0, 'trying': ...",1,0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4,"{'for': 1.0, 'barnes':1.0, 'at': 1.0, 'is': ...",1,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5,"{'right': 1.0, 'because':1.0, 'questions': 1.0, ...",1,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5,"{'like': 1.0, 'and': 1.0,'changes': 1.0, 'the': ...",1,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4,"{'in': 1.0, 'pages': 1.0,'out': 1.0, 'run': 1.0, ...",1,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4,"{'tracker': 1.0, 'now':1.0, 'its': 1.0, 'sti ...",1,0


In [27]:
selected_words

['awesome',
 'great',
 'fantastic',
 'amazing',
 'love',
 'horrible',
 'bad',
 'terrible',
 'awful',
 'wow',
 'hate']

In [28]:
products["hate"].sum()

945

In [29]:
train_data,test_data = products.random_split(.8, seed=0)

In [30]:
selected_words_model=tc.logistic_classifier.create(train_data,target="sentiments",features=selected_words,validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 11

...

| 4         | 5        | 0.208978     | 0.843257          | 0.841821            |

| 5         | 6        | 0.255936     | 0.843257          | 0.841821            |

| 6         | 7        | 0.301799     | 0.843257          | 0.841821            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

In [31]:
selected_words_model.summary() 

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 12
Number of examples             : 133448
Number of classes              : 2
Number of feature columns      : 11
Number of unpacked features    : 11

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : newton
Solver iterations              : 6
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.3201

Settings
--------
Log-likelihood                 : 55232.3513

Highest Positive Coefficients
-----------------------------
(intercept)                    : 1.4509
love                           : 1.3159
amazing                        : 0.8774
awesome                        : 0.8753
fantastic                      : 0.7485

Lowest Negative Coefficients
----------------------------
terrible                       : -1.936
hor

In [33]:
selected_words_model.evaluate(test_data)

{'accuracy': 0.8418208023060293,
 'auc': 0.6334051693600947,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        0        |   81  |
 |      0       |        0        |  141  |
 |      0       |        1        |  5187 |
 |      1       |        1        | 27895 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.9137213796717875,
 'log_loss': 0.41439735681789197,
 'precision': 0.8432077867118071,
 'recall': 0.9971046611381184,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+-----+-----+-------+------+
 | threshold | fpr | tpr |   p   |  n   |
 +-----------+-----+-----+-------+------+
 |    0.0    | 1.0 | 1.0 | 27976 | 5328 |
 |   1e-05   | 1.0 | 1.0 | 27976 | 5328 |
 |   2e-05   

In [34]:
sentiment_model.evaluate(test_data)

{'accuracy': 0.9176975738650012,
 'auc': 0.9342357833151299,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        1        |  1397 |
 |      1       |        0        |  1344 |
 |      0       |        0        |  3931 |
 |      1       |        1        | 26632 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.951057941255245,
 'log_loss': 0.3304787187241198,
 'precision': 0.9501587641371436,
 'recall': 0.9519588218472976,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+--------------------+--------------------+-------+------+
 | threshold |        fpr         |        tpr         |   p   |  n   |
 +-----------+--------------------+--------------------+-------+------+
 |    0.0

In [39]:
test_data.show()

Materializing SFrame

Warning: Skipping column 'word_count'. Unable to show columns of type 'dictionary'; only [int, float, str] can be shown.

Further warnings of unsupported type will be suppressed.

In [40]:
diaper_reviews=products[products["name"]=="Baby Trend Diaper Champ"]

In [41]:
diaper_reviews["predicted_sentiments"]=sentiment_model.predict(diaper_reviews,output_type="probability")

In [42]:
diaper_reviews

name,review,rating,word_count,sentiments,awesome
Baby Trend Diaper Champ,Ok - newsflash. Diapersare just smelly. We've ...,4,"{'convenient': 1.0,'more': 1.0, 'trash': ...",1,0
Baby Trend Diaper Champ,"My husband and I selectedthe Diaper ""Champ"" ma ...",1,"{'system': 1.0, 'try':1.0, 're': 1.0, 'still': ...",0,0
Baby Trend Diaper Champ,Excellent diaper disposalunit. I used it in ...,5,"{'nose': 1.0, 'for': 2.0,'investment': 1.0, ...",1,0
Baby Trend Diaper Champ,We love our diaper champ.It is very easy to use ...,5,"{'out': 1.0, 'pull': 1.0,'open': 1.0, 'pail': ...",1,0
Baby Trend Diaper Champ,Two girlfriends and twofamily members put me ...,5,"{'winter': 1.0,'outside': 1.0, 'day': ...",1,0
Baby Trend Diaper Champ,I waited to review thisuntil I saw how it ...,4,"{'mom': 1.0, 'my': 1.0,'empty': 2.0, 'poop': ...",1,0
Baby Trend Diaper Champ,I have had a diaper geniefor almost 4 years since ...,1,"{'yuck': 1.0, 'clean':1.0, 'all': 1.0, 'tra ...",0,0
Baby Trend Diaper Champ,I originally put thisitem on my baby registry ...,5,"{'price': 1.0,'suggestions': 1.0, ...",1,0
Baby Trend Diaper Champ,I am so glad I got theDiaper Champ instead of ...,5,"{'best': 1.0, 'that':1.0, 'handle': 1.0, ' ...",1,0
Baby Trend Diaper Champ,We had 2 diaper Genie'sboth given to us as a ...,4,"{'no': 1.0, 'regular':1.0, 'part': 1.0, ...",1,0


In [43]:
diaper_reviews=diaper_reviews.sort("predicted_sentiments",ascending=False)

In [47]:
diaper_reviews

name,review,rating,word_count,sentiments,awesome
Baby Trend Diaper Champ,I read a review belowthat can explain exactly ...,4,"{'key': 1.0, 'have': 1.0,'pieces': 1.0, 'betwe ...",1,0
Baby Trend Diaper Champ,I have never written areview for Amazon but I ...,5,"{'priceless': 1.0,'knows': 1.0, 'parent': ...",1,0
Baby Trend Diaper Champ,I originally put thisitem on my baby registry ...,5,"{'price': 1.0,'suggestions': 1.0, ...",1,0
Baby Trend Diaper Champ,Baby Luke can turn aclean diaper to a dirty ...,5,"{'around': 1.0, 'any':1.0, 't': 1.0, 'isn': ...",1,0
Baby Trend Diaper Champ,Diaper Champ or DiaperGenie? That was my ...,5,"{'either': 1.0, 'be':1.0, 't': 1.0, 'not': ...",1,0
Baby Trend Diaper Champ,I am one of those super-critical shoppers who ...,5,"{'hope': 1.0, 'make':1.0, 'slower': 1.0, ...",1,0
Baby Trend Diaper Champ,I LOOOVE this diaperpail! Its the easies ...,5,"{'buy': 1.0, 'product':1.0, 'recommend': 1.0, ...",1,0
Baby Trend Diaper Champ,"As a first time mother, Iwanted to get the best ...",5,"{'ll': 1.0, 'baby': 1.0,'recommended': 1.0, ' ...",1,0
Baby Trend Diaper Champ,I see that there arecomplaints of stinkiness ...,5,"{'very': 1.0, 'told':1.0, 'all': 1.0, ...",1,0
Baby Trend Diaper Champ,I have a 10 year olddaughter and an 8 month ...,5,"{'sorry': 1.0, 'be': 1.0,'you': 2.0, 'sell': 1.0, ...",1,0
